In [1]:
data_var = '2024-01-16'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9396,2024-01-16,Eua Nba,00:30,Los Angeles Lakers,Oklahoma City Thunder,2.15,1.82,237.5,1.89,2.04,1.5,2.05,1.89,EPoQPXfD,0.465116,0.549451,0.529101,0.490196,0.014567,0.4,0.8,NaN,NaN,237.456,46.686838,0.196613,187.410,54.832984,0.292583,210.756,252.854,224.54,207.36,0.0,0.0,0.0,0.0,0.117554,0.053978,0.057430,-0.65,-0.130,-8.846154,0.581236,0.4,-0.181236,-0.43,-0.086,-9.534884,0.548017,0.5,-0.048017
9397,2024-01-16,Eua Nba,21:30,Philadelphia 76ers,Denver Nuggets,1.94,2.00,228.5,1.88,2.05,-1.5,2.03,1.89,SSZ7lHXD,0.515464,0.500000,0.531915,0.487805,0.015464,0.2,0.4,NaN,NaN,191.810,75.503158,0.393635,152.880,18.572178,0.121482,155.686,173.374,173.60,159.84,0.0,0.0,0.0,0.0,0.021536,0.061175,0.050508,-0.50,-0.100,-9.400000,0.771892,0.7,-0.071892,1.02,0.204,4.901961,0.592226,0.7,0.107774
9398,2024-01-16,Eua Nba,23:00,Phoenix Suns,Sacramento Kings,1.57,2.65,241.5,1.83,2.04,-5.5,2.03,1.89,tWwCmymK,0.636943,0.377358,0.546448,0.490196,0.014301,0.6,0.2,NaN,NaN,203.772,77.075133,0.378242,220.140,101.960350,0.463161,160.272,235.294,179.40,401.86,0.0,0.0,0.0,0.0,0.361931,0.076740,0.050508,-1.25,-0.250,-2.280000,0.687756,0.5,-0.187756,-0.33,-0.066,-25.000000,0.546774,0.6,0.053226
9399,2024-01-16,Europa Eurocopa,16:00,Trento,Slask Wroclaw,1.13,5.46,159.5,1.80,1.86,-13.5,2.07,1.67,Oz3j35wB,0.884956,0.183150,0.555556,0.537634,0.068106,0.2,0.4,NaN,NaN,255.826,186.870935,0.730461,471.892,223.912906,0.474500,142.582,717.742,110.20,320.21,0.0,0.0,0.0,0.0,0.929218,0.023184,0.151253,0.15,0.030,4.333333,0.000000,0.0,0.000000,1.45,0.290,15.379310,0.000000,0.0,0.000000
9400,2024-01-16,China Cba,08:35,Guangdong,Qingdao,1.48,2.57,203.5,1.81,1.91,-7.5,2.07,1.65,badpJ8Am,0.675676,0.389105,0.552486,0.523560,0.064781,0.2,0.2,NaN,NaN,135.274,22.443237,0.165909,228.618,114.878439,0.502491,144.930,213.764,151.20,136.50,0.0,0.0,0.0,0.0,0.380616,0.038016,0.159669,1.42,0.284,1.690141,0.835039,1.0,0.164961,-0.04,-0.008,-196.250000,0.478968,0.4,-0.078968
9401,2024-01-16,China Cba,08:35,Ningbo Rockets,Shanghai,3.87,1.24,202.5,1.83,1.91,8.5,1.96,1.73,K8ctKles,0.258398,0.806452,0.546448,0.523560,0.064850,0.6,0.6,NaN,NaN,548.556,142.133047,0.259104,178.754,72.697234,0.406689,385.160,217.360,486.40,305.91,1.0,0.0,0.0,0.0,0.727863,0.030251,0.088149,-2.03,-0.406,-7.068966,0.302999,0.3,-0.002999,-0.83,-0.166,-1.445783,0.476106,0.4,-0.076106
9402,2024-01-16,Eua Ncaa,20:00,Western Carolina,Samford,1.91,1.91,157.5,1.91,1.80,-1.5,1.91,1.80,0Yc2vJqd,0.523560,0.523560,0.523560,0.555556,0.047120,0.0,0.0,NaN,NaN,124.512,24.448728,0.196356,116.260,11.647601,0.100186,0.000,259.270,93.80,107.20,0.0,0.0,0.0,0.0,0.000000,0.041931,0.041931,0.00,0.000,inf,0.000000,0.0,0.000000,-1.11,-0.222,-4.099099,0.000000,0.0,0.000000
9403,2024-01-16,Eua Ncaa,20:30,Xavier,Butler,1.45,2.88,149.5,1.83,1.83,-5.5,1.91,1.91,lKilgJQf,0.689655,0.347222,0.546448,0.546448,0.036877,0.6,0.4,NaN,NaN,183.170,35.746356,0.195154,224.596,78.565362,0.349807,153.234,225.794,195.75,337.41,0.0,0.0,0.0,0.0,0.467050,0.000000,0.000000,-1.37,-0.274,-1.642336,0.000000,0.0,0.000000,1.89,0.378,4.973545,0.000000,0.0,0.000000
9404,2024-01-16,Eua Ncaa,21:00,Akron,Western Michigan,1.09,8.20,144.5,1.83,1.83,-13.5,1.91,1.80,8lOeFURt,0.917431,0.121951,0.546448,0.546448,0.039382,0.4,1.0,NaN,NaN,103.364,16.902300,0.163522,239.150,110.781841,0.463232,102.030,316.844,80.56,178.76,0.0,0.0,0.0,0.0,1.082353,0.000000,0.041931,1.60,0.320,0.281250,0.000000,0.0,0.000000,0.05,0.0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Los Angeles Lakers,Oklahoma City Thunder,237.5,1.89,1.0000,Over
1,21:30,Eua Nba,Philadelphia 76ers,Denver Nuggets,228.5,1.88,1.0000,Over
2,23:00,Eua Nba,Phoenix Suns,Sacramento Kings,241.5,1.83,0.9998,Over
3,21:00,Eua Ncaa,Ohio,Central Michigan,145.5,1.91,0.8530,Over
4,21:00,Eua Ncaa,Tennessee,Florida,159.5,1.91,0.8444,Over
5,22:00,Eua Ncaa,Northern Illinois,Kent State,149.5,1.91,0.8343,Over
6,22:00,Eua Ncaa,SMU Mustangs,Temple,138.5,1.91,0.8674,Over
